<a href="https://colab.research.google.com/github/TXH2020/MainRepo/blob/main/Projects/Python%20SQL/Python_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Python with MYSQL:

In [ ]:
!apt-get update
!apt install mysql-server

In [ ]:
!sudo service mysql start

In [ ]:
!pip install mysql-connector-python

In [ ]:
#The things that I did to configure the mysql server
#Executed the following command: !mysql -u root
#After login typed: use mysql;
#Then executed the below command:
#ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'jtp12345';
#Then typed exit

In [ ]:
!mysql -u root

In [ ]:
#Python MySQL
import requests
response=requests.get("https://raw.githubusercontent.com/TXH2020/MainRepo/main/Projects/Python%20SQL/example.sql")
with open("mysql.sql",'wb') as file:
    file.write(response.content)

import mysql.connector as my
try:
 cnx=my.connect(user="root",host="localhost",password="jtp12345")
except my.Error as e:
 print(e)
else:
    file=open('mysql.sql','r',encoding='latin-1')
    s=file.read()
    file.close()
    l=s.split(';')
    cr=cnx.cursor()
    for i in l:
        try:
            cr.execute(i,multi=True)
        except my.Error as e:
            print(e.msg)
    cnx.commit()
    cnx.close()

try:
     cnx = my.connect(user="root",host="localhost",password="jtp12345")
except my.Error as e:
 print(e)
else:
 cr=cnx.cursor()
 print("******************")
 cr.execute("use python_mysql")
 cr.execute("Select *from books")    #Manipulate database
 k=0
 for i in cr:
     k=k+1
     print(i)
 print("No. of records=", k)
 print("******************")

 cr.execute("show columns from books")
 for i in cr:
     print(i[0])                         #Showing column names
 print("******************")

 add_book = ("INSERT INTO books"
                 "(id, title, isbn)"
                 "VALUES (%s, %s, %s)")
 book = (86,'Hello World',"1234567898784")
 cr.execute(add_book,book)            #Inserting record
 cr.execute("select count(id) from books")
 print("Number of records after inserting is=",cr.fetchall())
 print("*******************")
 cr.execute("delete from books where id=86")
 cr.execute("select count(id) from books")
 print("Number of records after deleting row with id=86 is=", cr.fetchall())
 cnx.close()
#Like this we can use any Mysql command in Python using mysql connector.

In [ ]:
!sudo service mysql stop

 * Stopping MySQL database server mysqld
   ...done.


##Python with NoSQL(MongoDB)

In [ ]:
!sudo apt-get install mongodb

In [ ]:
!sudo apt-get update

In [ ]:
!sudo service mongodb start

In [ ]:
#Python with mongodb nosql
import pandas as pd
import pymongo as pm
import numpy as np
df = pd.read_csv (r'https://raw.githubusercontent.com/TXH2020/MainRepo/main/Projects/Python%20SQL/listings.csv')
client = pm.MongoClient('mongodb://localhost:27017/')
mydb=client['somedatabase']
if 'somecollection' in mydb.list_collection_names():
    mydb['somecollection'].drop()
mycol=mydb['somecollection']
for i in range(df.shape[0]):
    r=dict(df.iloc[i])
    try:
        mycol.insert_one(r)
    except:
        # Python 2.7.10 on Windows and Pymongo are not forgiving
        # If you have foreign data types you have to convert them
        n = {}
        for k, v in r.items():
            if isinstance(v, np.int64):
                v = int(v)
            elif isinstance(v,np.float64):
                v=float(v)
            n[k]=v
        mycol.insert_one(n)
print("No of documents=",mycol.count_documents({}))
print("Document with id=2818")
print('---------------------------')
doc=mycol.find({'id':2818})
for i in doc:
    for j in i.items():
        print(j)
client.close()

In [ ]:
!sudo service mongodb stop

##Apache Cassandra:

In [ ]:
!wget https://dlcdn.apache.org/cassandra/4.0.6/apache-cassandra-4.0.6-bin.tar.gz

In [ ]:
!tar xzvf apache-cassandra-4.0.6-bin.tar.gz

In [ ]:
!apache-cassandra-4.0.6/bin/cassandra -R

In [ ]:
!pip3 install cassandra-driver

In [ ]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/TXH2020/MainRepo/main/Projects/Python%20SQL/data.csv",encoding="latin1",header=None)
df.columns=['id','title','isbn']
df['isbn']=df['isbn'].apply(lambda x:str(x))
df['title']=df['title'].apply(lambda x:x.replace("'",' '))

In [ ]:
from cassandra.cluster import Cluster
clstr=Cluster()
session=clstr.connect()
session.execute("""create keyspace mykeyspace with replication={
   'class': 'SimpleStrategy', 'replication_factor' : 1
};""")


In [ ]:
session=clstr.connect('mykeyspace')
qry= """
create table data (
   id int,
   name text,
   isbn text,
   primary key(id))"""
session.execute(qry)

In [ ]:
q="""insert into data (id,name,isbn) values """
for i in range(df.shape[0]):
  try: 
    session.execute(q+str(tuple(df.iloc[i])))
  except:
    pass

In [ ]:
q1="select count(*) from data"
row=session.execute(q1)
print("count = ",str(row.current_rows)[11:13])

In [ ]:
q2="select * from data;"
rows=session.execute(q2)
l=[]
for row in rows:
  l.append(list(row))

In [ ]:
l.sort(key=lambda x:x[0])

In [ ]:
print("Result of select command")
for i in l:
  print(i[0],i[1],i[2])

In [ ]:
#Kill the cassandra server
import psutil
# Iterate over all running process
for proc in psutil.process_iter():
    try:
        # Get process name & pid from process object.
        processName = proc.name()
        if(processName=='java'):
          proc.kill()
          break
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

##Python with Cloud SQL(Azure SQL DB)

In [ ]:
!pip install pyodbc

In [ ]:
#Python with Azure Cloud SQL. However one must create an SQL server in Azure which isn't free.
#Make sure that the database is created with sample data(AdventureWorksLT)

import pyodbc
server = 'tezserver.database.windows.net' #Enter your respective server name
database = 'somedatabase'                 #Enter your respective database name
username = 'Moto12'                       #Enter your respective usernname and password for SQL server
password = 'Hello123*'
driver= '{SQL Server}'

with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("select * from [SalesLT].[Product]")
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]))
            row = cursor.fetchone()